<a href="https://colab.research.google.com/github/talesoverfables/Q-NodE/blob/main/Phase-3/Sim_Room2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qutip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 12.5 MB/s eta 0:00:00


# User-based decryption

In [ ]:
import numpy as np
from qutip import basis, sigmax, Qobj

In [ ]:
def text_to_binary(text):
    binary_text = ''.join(format(ord(char), '08b') for char in text)
    return binary_text

In [ ]:
def bb84_encrypt(binary_text):
    alice_basis = np.random.randint(2, size=len(binary_text))
    alice_state = np.array([int(bit) for bit in binary_text])
    alice_qubits = []
    for i in range(len(binary_text)):
        if alice_basis[i] == 0:
            if alice_state[i] == 0:
                alice_qubits.append(basis(2, 0))
            else:
                alice_qubits.append(basis(2, 1))
        else:
            if alice_state[i] == 0:
                alice_qubits.append(sigmax() * basis(2, 0))
            else:
                alice_qubits.append(sigmax() * basis(2, 1))
    return alice_qubits, alice_basis

In [ ]:
def bb84_decrypt(alice_qubits, alice_basis, bob_basis):
    bob_qubits = [Qobj(qubit.full()) for qubit in alice_qubits]
    for i in range(len(bob_qubits)):
        if bob_basis[i] == 1:
            bob_qubits[i] = sigmax() * bob_qubits[i]
    bob_measurements = [np.random.choice([0, 1], p=[np.abs(q.full()[0, 0])**2, np.abs(q.full()[1, 0])**2]) for q in bob_qubits]
    return bob_measurements

In [ ]:
def generate_secret_key(alice_basis, bob_basis, alice_qubits):
    alice_key = []
    bob_key = []
    for i in range(len(alice_basis)):
        if alice_basis[i] == bob_basis[i]:
            alice_key.append(np.random.choice([0, 1], p=[np.abs(alice_qubits[i].full()[0, 0])**2, np.abs(alice_qubits[i].full()[1, 0])**2]))
            bob_key.append(np.random.choice([0, 1], p=[np.abs(alice_qubits[i].full()[0, 0])**2, np.abs(alice_qubits[i].full()[1, 0])**2]))
    return alice_key, bob_key

In [ ]:
def encrypt_message(binary_text, secret_key):
    encrypted_text = ''
    for i in range(len(binary_text)):
        encrypted_text += str(int(binary_text[i]) ^ secret_key[i % len(secret_key)])
    return encrypted_text

def decrypt_message(encrypted_text, secret_key):
    decrypted_text = ''
    for i in range(len(encrypted_text)):
        decrypted_text += str(int(encrypted_text[i]) ^ secret_key[i % len(secret_key)])
    return decrypted_text

In [ ]:
def main():
    text = input("Enter the text to be sent: ")
    binary_text = text_to_binary(text)
    alice_qubits, alice_basis = bb84_encrypt(binary_text)

    receiver = input("Enter the receiver (Bob1 or Bob2): ")

    if receiver == "Bob1":
        bob_basis = np.random.randint(2, size=len(binary_text))
        bob_measurements = bb84_decrypt(alice_qubits, alice_basis, bob_basis)
        alice_key, bob_key = generate_secret_key(alice_basis, bob_basis, alice_qubits)
        encrypted_text = encrypt_message(binary_text, alice_key)
        decrypted_text = decrypt_message(encrypted_text, bob_key)
        print("Decrypted text by Bob1:", ''.join([chr(int(decrypted_text[i*8:i*8+8], 2)) for i in range(len(decrypted_text)//8)]))

        # Bob2 receives a random message
        bob2_basis = np.random.randint(2, size=len(binary_text))
        bob2_measurements = bb84_decrypt(alice_qubits, alice_basis, bob2_basis)
        alice_key, bob2_key = generate_secret_key(alice_basis, bob2_basis, alice_qubits)
        random_message = "Something else"
        random_binary_message = text_to_binary(random_message)
        encrypted_text = encrypt_message(random_binary_message, alice_key)
        decrypted_text2 = decrypt_message(encrypted_text, bob2_key)
        print("Decrypted text by Bob2:", ''.join([chr(int(decrypted_text2[i*8:i*8+8], 2)) for i in range(len(decrypted_text2)//8)]))
    elif receiver == "Bob2":
        bob_basis = np.random.randint(2, size=len(binary_text))
        bob_measurements = bb84_decrypt(alice_qubits, alice_basis, bob_basis)
        alice_key, bob_key = generate_secret_key(alice_basis, bob_basis, alice_qubits)
        encrypted_text = encrypt_message(binary_text, alice_key)
        decrypted_text = decrypt_message(encrypted_text, bob_key)
        print("Decrypted text by Bob2:", ''.join([chr(int(decrypted_text[i*8:i*8+8], 2)) for i in range(len(decrypted_text)//8)]))

        # Bob1 receives a random message
        bob1_basis = np.random.randint(2, size=len(binary_text))
        bob1_measurements = bb84_decrypt(alice_qubits, alice_basis, bob1_basis)
        alice_key, bob1_key = generate_secret_key(alice_basis, bob1_basis, alice_qubits)
        random_message = "Something else"
        random_binary_message = text_to_binary(random_message)
        encrypted_text = encrypt_message(random_binary_message, alice_key)
        decrypted_text2 = decrypt_message(encrypted_text, bob1_key)
        print("Decrypted text by Bob1:", ''.join([chr(int(decrypted_text2[i*8:i*8+8], 2)) for i in range(len(decrypted_text2)//8)]))

In [ ]:
if __name__ == "__main__":
    main()

Enter the text to be sent: Hello, RVCE!
Enter the receiver (Bob1 or Bob2): Bob1
Decrypted text by Bob1: Hello, RVCE!
Decrypted text by Bob2: Something else
